<a href="https://colab.research.google.com/github/aivrm/ai/blob/main/%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_Top_4_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 앙상블 모델링으로 타이타닉 톱 4%

https://www.kaggle.com/code/yassineghouzam/titanic-top-4-with-ensemble-modeling


---
* 1 소개

* 2 로드 하고 데이터 체크
        2.1 데이터 로드
        2.2 Outlier 감지
        2.3 joining train 과 test 셋
        2.4 널과 없는(잘못된) 값 체크

* 3 특성 분석
        3.1 숫자 값
        3.2 범주 값

* 4 없는 값 채우기
        4.1 나이

* 5 특성 엔지니어링
        5.1 이름/타이틀
        5.2 가족 사이즈
        5.3 객실
        5.4 티켓

* 6 모델링
        6.1 단순 모델링
            6.1.1 상호 입증 모델
            6.1.2 최고의 모델을 위한 하이퍼파라미터 튜닝
            6.1.3 학습곡선 그리기
            6.1.4 트리 기반 분류기의 특성 중요도

        6.2 앙상블 모델
            6.2.1 결합 모델

        6.3 예측
            6.3.1 예측과 결과 제출
---


---
## 1. 소개

* 특성 분석
* 특성 엔지니어링
* 모델링
---

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

sns.set(style='white', context='notebook', palette='deep')

---
## 2. 데이터 로드 와 체크
---

### 2.1 데이터 로드

### 2.2 Outlier 감지

### 2.3 joining 훈련과 테스트 셋

### 2.4 널 과 빈 값 검사

---
## 3. 특성 분석
---

### 3.1 숫자값

### 3.2 범주값

---
## 4. 빈 값 채우기
---

### 4.1 나이

---
## 5. 특성 엔지니어링
---

### 5.1 이름/타이틀

### 5.2 가족 사이즈

### 5.3 객실

### 5.4 티켓

---
## 6. 모델링
---

### 6.1 단순 모델링





#### 6.1.2 최고의 모델을 위한 하이퍼마라미터 튜닝

#### 6.1.3 학습 곡선 그리기

#### 6.1.4 트리 기반 분류기의 특성 중요도

### 6.2 앙상블 모델링

#### 6.2.1 모델 결합

### 6.3 예측

#### 6.3.1 예측 과 결과 제출